This notebook serves as a sandbox for testing different hyperparameters for training the neural network. While this notebook can  be ran without a Nvidia GPU, the usage of one is reccomended as this notebook was written with using CUDA when running PyTorch in mind.

In [2]:
#Import neccessary libraries
import numpy as np
import pandas as pd
import torch as th

In [121]:
#Read data from dataset.csv and convert into numpy arrays
dataFrame = pd.read_csv("dataset.csv")
dataArray = dataFrame.to_numpy()
labels = dataArray[:, 34].astype(str)
features = np.delete(dataArray, 34, 1)

#Convert strings in labels array into ints and set as type 'int64'
#Additionally, create a list containing the string values respective to the int representations for easy conversion later on
intLabels = np.empty(4424, dtype = 'int64')
for i in range(4424):
    if(labels[i] == "Dropout"):
        intLabels[i] = 0
    elif(labels[i] == "Graduate"):
        intLabels[i] = 1
    elif(labels[i] == "Enrolled"):
        intLabels[i] = 2
labels = intLabels
labelsRefrence =["Dropout", "Graduate", "Enrolled"]

#Clean up features array by removing biased features, one-hot encoding, and standardization
features = np.delete(features, [0, 1, 2, 6, 7, 8, 9, 10, 11, 12, 15, 18], 1)

courseEncode = np.zeros((4424, 17), dtype = 'float32')
qualiEncode = np.zeros((4424, 17), dtype = 'float32')
for i in range(4424):
    course = features[i, 0]
    courseEncode[i, (course - 1)] = 1
    quali = features[i, 2]
    qualiEncode[i, (quali - 1)] = 1
hotFeatures = np.concatenate((courseEncode, qualiEncode), axis = 1)

boolFeatures = np.vstack((features[:, 1], features[:, 3], features[:, 4], features[:, 5])).astype('float32').T
unstdFeatures = np.delete(features, [0, 1, 2, 3, 4, 5], 1).astype('float32')
stdFeatures = np.empty((4424, 16), dtype = 'float32')
for i in range(16):
    mean = np.mean(unstdFeatures[:, i])
    std = np.std(unstdFeatures[:, i])
    for j in range(4424):
        stdFeatures[j, i] = (unstdFeatures[j, i] - mean) / std

features = np.concatenate((stdFeatures, boolFeatures, hotFeatures), axis = 1)

#Dataset class used for creating datasets out of the training and validation data
class Dataset(th.utils.data.Dataset):
    def __init__(self, features, labels):
        self.features = features
        self.labels = labels
    
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        return self.features[idx, :], self.labels[idx]
    
#Function for creating dataloaders using the training and validation sets made with getSets
#The dataloader passes samples in batches of 79 as it is a factor of both 1106 and 3318
#which are the number of entries in the validation and training sets respectively
def buildLoaders(tFeatures, tLabels, vFeatures, vLabels):
    tDataset = Dataset(tFeatures, tLabels)
    vDataset = Dataset(vFeatures, vLabels)
    tLoader = th.utils.data.DataLoader(tDataset, batch_size = 79)
    vLoader = th.utils.data.DataLoader(vDataset, batch_size = 79,)
    return tLoader, vLoader

#NeuralNetwork class used for the building the model
#Neural Network sepcific hyperparameters are edited directly in this class
class NeuralNetwork(th.nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = th.nn.Flatten()
        self.linearReLUStack = th.nn.Sequential(
            th.nn.Linear(54, 27),
            th.nn.Dropout(0.2),
            th.nn.ReLU(),
            th.nn.Linear(27, 3)
        )
        
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linearReLUStack(x)
        return logits
    
#Select the device to be used for training the model and send the model to it
#Uses GPU if a Nvdia GPU is detected and PyTorch was installed with the CUDA platform, uses CPU otherwise
device = "cuda" if th.cuda.is_available() else "cpu"
model = NeuralNetwork().to(device)

#Define a loss function and an optimizer for training the model
lossFunc = th.nn.CrossEntropyLoss()
optimizer = th.optim.SGD(model.parameters(), lr = 0.005, momentum = 0.99, weight_decay = 0.001)

#Define training function
def train(tLoader, model, lossFunc, optimizer):
    size = len(tLoader.dataset)
    model.train()
    for batch, (X, y) in enumerate(tLoader):
        X, y = X.to(device), y.to(device)
        
        #Compute Prediction Error
        prediction = model(X)
        loss = lossFunc(prediction, y)
        
        #Backpropagate
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

#Define testing fuction
def test(vLoader, model, lossFunc):
    size = len(vLoader.dataset)
    numBatches = len(vLoader)
    model.eval()
    tLoss, correct = 0, 0
    with th.no_grad():
        for X, y in vLoader:
            X, y = X.to(device), y.to(device)
            prediction = model(X)
            tLoss += lossFunc(prediction, y).item()
            correct += (prediction.argmax(1) == y).type(th.float).sum().item()
    tLoss /= numBatches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {tLoss:>8f} \n")

#Create datasets for trianing data and validation data
pIndex = np.random.permutation(4424)
vNum = int(4424 * 0.2)
vIndex = pIndex[:vNum]
tIndex = pIndex[vNum:]
tFeatures = features[tIndex]
tLabels = labels[tIndex]
vFeatures = features[vIndex]
vLabels = labels[vIndex]

#Train the model for the given number of epochs and print out the training and testing error
epochs = 12
tLoader, vLoader = buildLoaders(tFeatures, tLabels, vFeatures, vLabels)
for i in range(epochs):
    print(f"Epoch {i+1}\n-------------------------------")
    train(tLoader, model, lossFunc, optimizer)
    test(vLoader, model, lossFunc)


Epoch 1
-------------------------------
Test Error: 
 Accuracy: 69.1%, Avg loss: 0.802400 

Epoch 2
-------------------------------
Test Error: 
 Accuracy: 72.4%, Avg loss: 0.680617 

Epoch 3
-------------------------------
Test Error: 
 Accuracy: 74.7%, Avg loss: 0.638255 

Epoch 4
-------------------------------
Test Error: 
 Accuracy: 77.8%, Avg loss: 0.604180 

Epoch 5
-------------------------------
Test Error: 
 Accuracy: 77.4%, Avg loss: 0.593639 

Epoch 6
-------------------------------
Test Error: 
 Accuracy: 78.2%, Avg loss: 0.588287 

Epoch 7
-------------------------------
Test Error: 
 Accuracy: 79.0%, Avg loss: 0.583763 

Epoch 8
-------------------------------
Test Error: 
 Accuracy: 78.2%, Avg loss: 0.572082 

Epoch 9
-------------------------------
Test Error: 
 Accuracy: 79.1%, Avg loss: 0.567059 

Epoch 10
-------------------------------
Test Error: 
 Accuracy: 79.8%, Avg loss: 0.563783 

Epoch 11
-------------------------------
Test Error: 
 Accuracy: 79.6%, Avg los

Accuracy hits 80% with lr = 0.005, momentum = 0.99, and weight_decay = 0.01    
1 hidden layer of size 27, dropout of 0.2    
epoch 12